In [21]:
from requests_html import AsyncHTMLSession
import pandas as pd
import numpy as np
import cbb_data_scrape as scrape
import warnings
warnings.filterwarnings("ignore")


### PARAMS ###

DATE = "2021-11-17"


###
####### 1) Haslam Metrics ######
##

print("Starting for date: " + DATE)
print("")
print("Scraping Haslam metrics...")
asession = AsyncHTMLSession()
r = await asession.get("https://haslametrics.com/ratings.php")
await r.html.arender()

home_teams = pd.DataFrame()
away_teams = pd.DataFrame()
home_scores = pd.DataFrame()
away_scores = pd.DataFrame()

# Scrape favorites
for a in r.html.find('.scoreproj1'):
    gid = a.attrs['id'].replace("_sc","")
    gid = gid[:gid.rindex("_")]
    if(not(a.attrs['id'].endswith("_sc"))):
        value = ''.join([i for i in a.text if not i.isdigit()]).strip()
        home_teams = home_teams.append({'game_id':gid,'team':value},ignore_index=True)
    else:
        value = a.text
        home_scores = home_scores.append({'game_id':gid,'score':value},ignore_index=True)

# Scrape dogs
for a in r.html.find('.scoreproj2'):
    gid = a.attrs['id'].replace("_sc","")
    gid = gid[:gid.rindex("_")]
    if(not(a.attrs['id'].endswith("_sc"))):
        value = ''.join([i for i in a.text if not i.isdigit()]).strip()
        away_teams = away_teams.append({'game_id':gid,'team':value},ignore_index=True)
    else:
        value = a.text
        away_scores = away_scores.append({'game_id':gid,'score':value},ignore_index=True)

##Convert team names to KP standard
haslam_kp_map = pd.read_csv("Hasla_Kenpom_Mapping.csv");
home_teams = pd.merge(home_teams,haslam_kp_map,on='team')
away_teams = pd.merge(away_teams,haslam_kp_map,on='team')
teams = pd.merge(home_teams,away_teams,on='game_id',suffixes=('_home','_away'))

## Merge, drop empty
scores = pd.merge(home_scores,away_scores,on='game_id',suffixes=('_home','_away'))
haslam = pd.merge(teams,scores,on='game_id')
haslam.replace('', np.nan, inplace=True)
haslam.dropna(inplace=True)

print("Done for " + str(len(haslam)) + " games")

##### 2) Pull odds data from VegasInsiders ######

print("Getting latest spreads...")
odds = scrape.scrape_odds(DATE)

# Convert team names to KP standard
kp_odds_map=pd.read_csv("kenpom_team_name_convert.csv")
odds = pd.merge(kp_odds_map,odds,on="ODDS_NAME",how="right")
kp_odds_map['OPPONENT']=kp_odds_map['ODDS_NAME']
odds = pd.merge(kp_odds_map[['OPPONENT','KP_NAME']],odds,on="OPPONENT",how="right",suffixes=('','_OPP'))
odds['KP_NAME_home']=odds['KP_NAME']
odds['KP_NAME_away']=odds['KP_NAME']

#### 3) Calculate Bets

# merge Haslam with odds
m = pd.merge(haslam,odds,on='KP_NAME_home',suffixes=('','_home'))[['DATE','TIME','KP_NAME_home','KP_NAME_away','score_away','score_home','OPEN_SPREAD','CLOSE_SPREAD','HOME']]
m.rename(columns={'KP_NAME_home':'haslam_favorite','KP_NAME_away':'haslam_dog','score_away':'haslam_dog_score','score_home':'haslam_fav_score'},inplace=True)

# Find the bets
m['haslam_fav_spread']=m['haslam_dog_score'].astype(float)-m['haslam_fav_score'].astype(float)
m['disparity']=abs(m.CLOSE_SPREAD- m.haslam_fav_spread)
m['haslam_bet']=m.haslam_dog
m.loc[m.CLOSE_SPREAD- m.haslam_fav_spread > 0,'haslam_bet']=m.haslam_favorite

### 4) Get game results

game_results = scrape.get_scores(DATE)
game_results['TEAM'] = game_results['TEAM'].str.replace(r"[\"\\,]", '')
game_results['haslam_favorite']=game_results['TEAM']
game_results['haslam_dog']=game_results['TEAM']

f = pd.merge(m,game_results[['haslam_favorite','SCORE']],on='haslam_favorite',how='left')
f = pd.merge(f,game_results[['haslam_dog','SCORE']],on='haslam_dog',how='left')
f.rename(columns={'SCORE_x':'haslam_fav_result','SCORE_y':'haslam_dog_result'},inplace=True)


## Get bet results for completed games
f['margin']= f.haslam_fav_result.astype(float)-f.haslam_dog_result.astype(float)
f.loc[f['margin']+f['CLOSE_SPREAD']<0,'ats_winner']=f.haslam_dog
f.loc[f['margin']+f['CLOSE_SPREAD']>0,'ats_winner']=f.haslam_favorite
f['haslam_bet_result']=0
f.loc[f['ats_winner']==f['haslam_bet'],'haslam_bet_result']=1
f['haslam_error']=abs(f.haslam_fav_spread+f.margin)
f['vegas_error']=abs(f.CLOSE_SPREAD+f.margin)
f['haslam_error_vs_vegas']=f.haslam_error - f.vegas_error

# Some formatting
f['result']=f['haslam_favorite'].astype(str)+"("+f['haslam_fav_result'].astype(str)+") - "+f['haslam_dog'].astype(str)+"("+f['haslam_dog_result'].astype(str)+")"
f.loc[f.haslam_fav_result.isnull(),'result']="-"
f.loc[f.haslam_fav_result.isnull(),'ats_winner']="-"
#Save all the data down to CSV
f.to_csv("HaslamResults_"+DATE)

##### 4) Summarize results
print("")

def print_summary_for_min_disp(min_disp,f):
    if(len(f.query("disparity>"+str(min_disp)).dropna())>0):
        print("Haslam ATS (disp>"+str(min_disp)+"): " + str(f.dropna().query("disparity>"+str(min_disp)).haslam_bet_result.sum()) + "-" + str(len(f.dropna().query("disparity>"+str(min_disp))) - f.dropna().query("disparity>"+str(min_disp)).haslam_bet_result.sum()))
        print("Haslam ATS (disp>"+str(min_disp)+") %: " + str(round(f.dropna().query("disparity>"+str(min_disp)).haslam_bet_result.mean(),3)))
        print("Haslam MAE (disp>"+str(min_disp)+") %: " + str(round(f.dropna().query("disparity>"+str(min_disp)).haslam_error.mean(),3)))
        print("Vegas MAE (disp>"+str(min_disp)+") %: " + str(round(f.dropna().query("disparity>"+str(min_disp)).vegas_error.mean(),3)))
        print("Games remaining: " + str(len(f.query("disparity>"+str(min_disp))) - len(f.query("disparity>"+str(min_disp)).dropna())))
        print("")

print("---- Today's Performance -----")
print_summary_for_min_disp(0,f)
print_summary_for_min_disp(3,f)
print_summary_for_min_disp(5,f)

"""
'DATE','TIME','haslam_favorite','haslam_dog','haslam_dog_score','haslam_fav_score','OPEN_SPREAD','CLOSE_SPREAD','disparity','haslam_bet',,'ats_winner'
"""

print("")
print("")
f.sort_values(by='disparity',ascending=False)[['DATE','TIME','haslam_favorite','haslam_dog','haslam_fav_score','haslam_dog_score','OPEN_SPREAD','CLOSE_SPREAD','disparity','haslam_bet','ats_winner','result']]


Starting for date: 2021-11-17

Scraping Haslam metrics...
Done for 41 games
Getting latest spreads...
Number of games: 41


---- Today's Performance -----
Haslam ATS (disp>0): 1-1
Haslam ATS (disp>0) %: 0.5
Haslam MAE (disp>0) %: 8.01
Vegas MAE (disp>0) %: 6.0
Games remaining: 39

Haslam ATS (disp>3): 1-0
Haslam ATS (disp>3) %: 1.0
Haslam MAE (disp>3) %: 2.62
Vegas MAE (disp>3) %: 1.5
Games remaining: 17





,DATE,TIME,haslam_favorite,haslam_dog,haslam_fav_score,haslam_dog_score,OPEN_SPREAD,CLOSE_SPREAD,disparity,haslam_bet,ats_winner,result
32,11/17,7:00 PM,Wake Forest,Charleston Southern,73.49,64.23,-19.5,-19.5,10.24,Charleston Southern,-,-
26,11/17,10:00 PM,Pepperdine,UC Davis,78.98,70.26,-1.0,-0.0,8.72,Pepperdine,-,-
27,11/17,9:00 PM,Texas Southern,Air Force,72.71,62.79,-2.0,-2.5,7.42,Texas Southern,-,-
31,11/17,5:30 PM,UTSA,IUPUI,80.44,64.26,-11.0,-9.5,6.68,UTSA,-,-
4,11/17,10:30 PM,UCLA,North Florida,79.69,59.83,-26.5,-25.5,5.64,North Florida,-,-
13,11/17,10:00 PM,Saint Mary's,Bellarmine,72.31,61.57,-17.0,-15.5,4.76,Bellarmine,-,-
12,11/17,10:00 PM,Loyola Marymount,Little Rock,77.69,60.56,-15.0,-12.5,4.63,Loyola Marymount,-,-
29,11/17,7:00 PM,Harvard,Albany,72.79,70.62,-5.5,-6.5,4.33,Albany,-,-
18,11/17,6:00 PM,Rhode Island,Boston College,79.67,70.01,-5.5,-5.5,4.16,Rhode Island,-,-
40,11/17,2:00 PM,Denver,Texas A&M Corpus Chris,72.48,71.86,1.5,3.5,4.12,Denver,Denver,Denver(67) - Texas A&M Corpus Chris(69)


In [23]:
## Kenpom Evaluator ##
# Run it after Haslam.  Showing for finished games only

### PARAMS ###

DATE = "2021-11-17"


###
f = pd.read_csv("HaslamResults_"+DATE)

kp_preds = scrape.get_kp_predictions(DATE)
kp_preds['KP_PRED']=kp_preds.SCORE
kp_preds['KP_PRED_OPP']=kp_preds.OPPONENT_SCORE
kp_preds['haslam_favorite']=kp_preds.TEAM
m2 = pd.merge(f,kp_preds[['haslam_favorite','KP_PRED','KP_PRED_OPP']],on='haslam_favorite',how='left')
kp_preds['haslam_dog']=kp_preds.TEAM
m2 = pd.merge(m2,kp_preds[['haslam_dog','KP_PRED','KP_PRED_OPP']],on='haslam_dog',how='left')
m2['KP_PRED']=m2['KP_PRED_x']
m2['KP_PRED_OPP']=m2['KP_PRED_OPP_x']
m2['KP_SPREAD']=-(m2.KP_PRED - m2.KP_PRED_OPP)
m2['kp_error']=abs(m2.KP_SPREAD+f.margin)

m2['kp_disparity']=abs(m2.CLOSE_SPREAD- m2.KP_SPREAD)
m2['KP_BET']=m2.haslam_dog
m2.loc[m2.CLOSE_SPREAD- m2.KP_SPREAD > 0,'KP_BET']=m2.haslam_favorite
m2['kp_bet_result']=0
m2.loc[m2['ats_winner']==m2['KP_BET'],'kp_bet_result']=1

m2.drop(columns={'KP_PRED_x','KP_PRED_OPP_x','KP_PRED_y','KP_PRED_OPP_y'},inplace=True)

def print_summary_for_min_disp(min_disp,f):
    print("KP ATS (disp>"+str(min_disp)+"): " + str(f.dropna().query("kp_disparity>"+str(min_disp)).kp_bet_result.sum()) + "-" + str(len(f.dropna().query("kp_disparity>"+str(min_disp))) - f.dropna().query("kp_disparity>"+str(min_disp)).kp_bet_result.sum()))
    print("KP ATS (disp>"+str(min_disp)+") %: " + str(round(f.dropna().query("kp_disparity>"+str(min_disp)).kp_bet_result.mean(),3)))
    print("KP MAE (disp>"+str(min_disp)+"): " + str(round(f.dropna().query("kp_disparity>"+str(min_disp)).kp_error.mean(),3)))
    print("KP MAE (disp>"+str(min_disp)+"): " + str(round(f.dropna().query("kp_disparity>"+str(min_disp)).vegas_error.mean(),3)))
    print("Games remaining: " + str(len(f.query("kp_disparity>"+str(min_disp))) - len(f.query("kp_disparity>"+str(min_disp)).dropna())))
    print("")

print_summary_for_min_disp(0,m2)
print_summary_for_min_disp(3,m2)
print_summary_for_min_disp(5,m2)
#[['DATE','TIME','haslam_favorite','haslam_dog','haslam_fav_score','haslam_dog_score','OPEN_SPREAD','CLOSE_SPREAD','disparity','haslam_bet','ats_winner','result']]
m2.dropna()[['DATE','TIME','result','kp_disparity','KP_BET','ats_winner','kp_bet_result']]





KP ATS (disp>0): 2-0
KP ATS (disp>0) %: 1.0
KP MAE (disp>0): 6.0
KP MAE (disp>0): 6.0
Games remaining: 0

KP ATS (disp>3): 1-0
KP ATS (disp>3) %: 1.0
KP MAE (disp>3): 3.0
KP MAE (disp>3): 1.5
Games remaining: 0

KP ATS (disp>5): 0-0
KP ATS (disp>5) %: nan
KP MAE (disp>5): nan
KP MAE (disp>5): nan
Games remaining: 0



,DATE,TIME,result,kp_disparity,KP_BET,ats_winner,kp_bet_result
20,11/17,1:00 PM,Texas A&M(73) - Houston Baptist(39),1.5,Texas A&M,Texas A&M,1
40,11/17,2:00 PM,Denver(67) - Texas A&M Corpus Chris(69),4.5,Denver,Denver,1
